In [15]:
#Implementation of the protocol from https://arxiv.org/pdf/2007.13126.pdf
#Honest verifier and fcg
#Advantage for the honest verifier and fcg setting : generator testing completely suffices
#erstmal completely honest und benchmark, wie quelle reagiert, also reihenfolge der tests egal - 
#wir sparen uns die auswahl aus 2n*n_test vielen Systemen
#alternativer approach : random number generator, der entscheidet, ob in der iteration noisy state
#oder idealer graph state gewählt wird

%matplotlib inline
import numpy as np
# Importing standard Qiskit libraries and configuring account
import qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math as math
# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

simulator2 = Aer.get_backend('statevector_simulator')

adj_matrix = [[0,1,1,1],[1,0,0,0],[1,0,0,0],[1,0,0,0]] #star 4

def sum_string_dict(dictionary, counter=0):
    """Sum the values of the keys with an even number of ones in an dictionary.

    Keyword arguments:
    dictionary -- dictionary with binary integers as string as keys and integers as values '011001': 42
    counter    -- start counter
    """
    for key_string, value_int in dictionary.items():
        condition = 0
        for bit in key_string:
            if bit == '1':
                condition += 1
        if (condition % 2) == 0:
            counter += value_int
    return counter

def not_sum_string_dict(dictionary, counter=0):
    """Sum the values of the keys with an even number of ones in an dictionary.

    Keyword arguments:
    dictionary -- dictionary with binary integers as string as keys and integers as values '011001': 42
    counter    -- start counter
    """
    for key_string, value_int in dictionary.items():
        condition = 0
        for bit in key_string:
            if bit == '1':
                condition += 1
        if (condition % 2) == 1:#aenderung hier : wegen -1 im stabilizer
            counter += value_int
    return counter

def stabilizer_test(circuit,num_qubits,alpha):
    for i in range(0,num_qubits):
        if i == alpha:
            circuit.h(i)
            circuit.measure(i,i)
        if adj_matrix[alpha][i] == 0:
            pass
        if adj_matrix[alpha][i] == 1:
            circuit.measure(i,i)
    return(circuit)
gamma = 4
def stabilizer_test_with_depolarizing_noise_line(noise_param,J,m):
    #countslist = [] #hilfsvariable um die ergebnisse zu checken
    n_pass = 0
    num_tests = math.ceil(m*gamma**4*math.log(gamma))
    num_qubits = J
    for alpha in range(J):
        #for beta in range(num_tests):
            ### white noise model/depolarizing channel
        graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
        new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
        graph_state_circuit.add_register(new_creg)
        graph_state_circuit.barrier()
        qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
        noise_model = NoiseModel()
        error = depolarizing_error(noise_param,num_qubits=1) #setze num_qubits auf 1, damit die 1-qubit-gatter mitspielen
        noise_model.add_all_qubit_quantum_error(error,'h')
        basis_gates = noise_model.basis_gates
        graph_state_circuit = stabilizer_test(graph_state_circuit,num_qubits,alpha)
        simulator = Aer.get_backend('qasm_simulator')
        shots = num_tests
        result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
        counts = result.get_counts(graph_state_circuit)
            #countslist.append(counts)
        n_pass += sum_string_dict(counts)
        
        
    ######## From here : comment/uncomment code to measure additional stabilizers with noise ################    
    #first additional stabilizer YYZZ
    #graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
    #new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
    #graph_state_circuit.add_register(new_creg)
    #graph_state_circuit.barrier()
    #qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
    #noise_model = NoiseModel()
    #error = depolarizing_error(noise_param,num_qubits=1) #setze num_qubits auf 1, damit die 1-qubit-gatter mitspielen
    #noise_model.add_all_qubit_quantum_error(error,'h')
    #basis_gates = noise_model.basis_gates
    #graph_state_circuit.sdg(0)
    #graph_state_circuit.h(0)
    #graph_state_circuit.measure(0,0)
    #graph_state_circuit.sdg(1)
    #graph_state_circuit.h(1)
    #graph_state_circuit.measure(1,1)
    #graph_state_circuit.measure(2,2)
    #graph_state_circuit.measure(3,3)
    #simulator = Aer.get_backend('qasm_simulator')
    #shots = num_tests
    #result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
    #counts = result.get_counts(graph_state_circuit)
            #countslist.append(counts)
    #n_pass += sum_string_dict(counts)
    #second additional stabilizer -YZYZ
    #graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
    #new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
    #graph_state_circuit.add_register(new_creg)
    #graph_state_circuit.barrier()
    #qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
    #noise_model = NoiseModel()
    #error = depolarizing_error(noise_param,num_qubits=1) #setze num_qubits auf 1, damit die 1-qubit-gatter mitspielen
    #noise_model.add_all_qubit_quantum_error(error,'h')
    #basis_gates = noise_model.basis_gates
    #graph_state_circuit.sdg(0)
    #graph_state_circuit.h(0)
    #graph_state_circuit.measure(0,0)
    #graph_state_circuit.measure(1,1)
    #graph_state_circuit.sdg(2)
    #graph_state_circuit.h(2)
    #graph_state_circuit.measure(2,2)
    #graph_state_circuit.measure(3,3)
    #simulator = Aer.get_backend('qasm_simulator')
    #shots = num_tests
    #result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
    #counts = result.get_counts(graph_state_circuit)
            #countslist.append(counts)
    #n_pass += sum_string_dict(counts)
    ##third additional stabilizer YZZY
    #graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
    #new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
    #graph_state_circuit.add_register(new_creg)
    #graph_state_circuit.barrier()
    #qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
    #noise_model = NoiseModel()
    #error = depolarizing_error(noise_param,num_qubits=1) #setze num_qubits auf 1, damit die 1-qubit-gatter mitspielen
    #noise_model.add_all_qubit_quantum_error(error,'h')
    #basis_gates = noise_model.basis_gates
    #graph_state_circuit.sdg(0)
    #graph_state_circuit.h(0)
    #graph_state_circuit.measure(0,0)
    #graph_state_circuit.measure(1,1)
    #graph_state_circuit.measure(2,2)
    #graph_state_circuit.sdg(3)
    #graph_state_circuit.h(3)
    #graph_state_circuit.measure(3,3)
    #simulator = Aer.get_backend('qasm_simulator')
    #shots = num_tests
    #result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
    #counts = result.get_counts(graph_state_circuit)
            #countslist.append(counts)
    #n_pass += sum_string_dict(counts)
    #fourth additional stabilizer IXXI
    #graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
    #new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
    #graph_state_circuit.add_register(new_creg)
    #graph_state_circuit.barrier()
    #qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
    #noise_model = NoiseModel()
    #error = depolarizing_error(noise_param,num_qubits=1) #setze num_qubits auf 1, damit die 1-qubit-gatter mitspielen
    #noise_model.add_all_qubit_quantum_error(error,'h')
    #basis_gates = noise_model.basis_gates
    #graph_state_circuit.h(1)
    #graph_state_circuit.measure(1,1)
    #graph_state_circuit.h(2)
    #graph_state_circuit.measure(2,2)
    #simulator = Aer.get_backend('qasm_simulator')
    #shots = num_tests
    #result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
    #counts = result.get_counts(graph_state_circuit)
            #countslist.append(counts)
    #n_pass += sum_string_dict(counts)
    return n_pass/(num_tests*gamma)



ibmqfactory.load_account:WARNING:2021-07-18 17:10:31,905: Credentials are already in use. The existing account in the session will be replaced.


In [17]:
noise_parameter = [0.01 for i in range(20)]

percentage_passed = [stabilizer_test_with_depolarizing_noise_line(noise_param,4,32) for noise_param in noise_parameter]

print(percentage_passed)
print(np.mean(percentage_passed),np.std(percentage_passed))

[0.9860878753191864, 0.9855155410759884, 0.9845910011446685, 0.9869023509729682, 0.9859998238971559, 0.9856696310645416, 0.9867702738399226, 0.9869023509729682, 0.9861979395967245, 0.986396055296293, 0.9860438496081712, 0.9865501452848463, 0.9865941709958616, 0.9856696310645416, 0.9855155410759884, 0.9860658624636788, 0.9856256053535265, 0.9880250066038566, 0.9858677467641103, 0.9859117724751255]
0.9861451087435062 0.0006930169944120165
